In [2]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

e:\Projects\LLM\llm-playground\Project-1\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
openai = OpenAI()

# How Gradio ChatInterface Manages History

## Visual Flow Diagram

```
┌─────────────────────────────────────────────────────────────┐
│                    USER SENDS MESSAGE                        │
│                    "Hello, how are you?"                     │
└───────────────────────┬─────────────────────────────────────┘
                        │
                        ▼
┌─────────────────────────────────────────────────────────────┐
│              GRADIO ChatInterface INTERCEPTS                 │
│                                                              │
│  1. Takes the new message: "Hello, how are you?"            │
│  2. Retrieves stored history from memory:                   │
│     [                                                         │
│       {"role": "user", "content": "Hi"},                     │
│       {"role": "assistant", "content": "Hello!"}             │
│     ]                                                         │
│  3. Calls your function: chat(message, history)             │
└───────────────────────┬─────────────────────────────────────┘
                        │
                        ▼
┌─────────────────────────────────────────────────────────────┐
│              YOUR FUNCTION RECEIVES:                         │
│                                                              │
│  message = "Hello, how are you?"                            │
│  history = [                                                 │
│    {"role": "user", "content": "Hi"},                       │
│    {"role": "assistant", "content": "Hello!"}                │
│  ]                                                           │
│                                                              │
│  You build:                                                 │
│  messages = history + [{"role": "user", "content": message}]│
│  = [                                                         │
│      {"role": "user", "content": "Hi"},                      │
│      {"role": "assistant", "content": "Hello!"},             │
│      {"role": "user", "content": "Hello, how are you?"}      │
│    ]                                                         │
└───────────────────────┬─────────────────────────────────────┘
                        │
                        ▼
┌─────────────────────────────────────────────────────────────┐
│              YOU CALL OPENAI API                             │
│  openai.chat.completions.create(messages=messages)          │
└───────────────────────┬─────────────────────────────────────┘
                        │
                        ▼
┌─────────────────────────────────────────────────────────────┐
│              YOU RETURN RESPONSE                             │
│  return "I'm doing well, thank you!"                        │
└───────────────────────┬─────────────────────────────────────┘
                        │
                        ▼
┌─────────────────────────────────────────────────────────────┐
│        GRADIO AUTOMATICALLY UPDATES HISTORY                  │
│                                                              │
│  New history stored:                                         │
│  [                                                           │
│    {"role": "user", "content": "Hi"},                        │
│    {"role": "assistant", "content": "Hello!"},               │
│    {"role": "user", "content": "Hello, how are you?"},       │
│    {"role": "assistant", "content": "I'm doing well..."}    │
│  ]                                                           │
│                                                              │
│  This history will be passed to your function                │
│  on the NEXT user message automatically!                     │
└─────────────────────────────────────────────────────────────┘
```

## Key Points:

1. **You DON'T manage history** - Gradio does it automatically
2. **History is cumulative** - Each call includes ALL previous messages
3. **Format is automatic** - Gradio stores it in OpenAI format for you
4. **You just append** - Add current message to history and call API
5. **Return value becomes history** - Your response is automatically added to history

In [17]:
def chat(message, history):
    # In Gradio 6.1.0, history is already in messages format (list of dicts with "role" and "content")
    # message is a string, history is a list of openai-style dictionaries
    
    # Build messages list for OpenAI API
    messages = []
    # Add history (already in correct format)
    messages.extend(history)
    # Add current user message
    messages.append({"role": "user", "content": message})
    
    # Call OpenAI API
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=messages
    )
    
    return response.choices[0].message.content

gr.ChatInterface(fn=chat).launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7871
* To create a public link, set `share=True` in `launch()`.
